In [ ]:
from pathlib import Path
from bressen import BasisGegevens, read_files
from bressen.kering import get_kering_geometry
from bressen.styles import add_styles_to_geopackage
from bressen.geometries import get_closest_feature, project_point, get_containing_feature, get_offsets
import geopandas as gpd
from shapely.geometry import LineString, Point

## Paden naar gegevens
Gegevens zijn aangeleverd in verschillende directories:
- 01_AGV_Dijktrajecten_Legger: dijktrajecten van waterschap Amstel Gooi en Vecht
- 02_ARK_Kering: kering van het Amsterdam-Rijnkanaal
- 03_Doorbraaklocaties: de locaties van de doorbraken
- 04_Watervlakken: alle beschikbare watervlakken

BASISGEGEVENS_GPKG en BRESSEN_GPKG verwijzen naar de basisgegevens (keringen, peil- en watervlakken) en het bressen-bestand

In [ ]:
DATA_DIR = Path(r"d:\projecten\D2405.waternet.bressen\01.gegevens")
BASISGEGEVENS_GPKG = DATA_DIR / "basisgegevens.gpkg"
BRESSEN_GPKG = DATA_DIR / "bressen.gpkg"

## Inlezen
Inlezen van basisgegevens en bressen

In [ ]:
# %% Inlezen basisgegevens vanuit GPKG
basis_gegevens = BasisGegevens.from_gpkg(BASISGEGEVENS_GPKG)

# Inlezen bressen vanuit meerdere bestanden
bressen_gdf = read_files([
    DATA_DIR.joinpath("03_Doorbraaklocaties", "240122_breslocaties_gelijk_groter_1000.shp"),
    DATA_DIR.joinpath("03_Doorbraaklocaties", "240212_breslocaties_kleiner_1000_angle.shp"),
    DATA_DIR.joinpath("03_Doorbraaklocaties", "240212_breslocaties_Waterkeringen_Blaeu_ARK.shp"),
    ])


## Verwerken Bressen
We hanteren de volgende instellingen
`x1`: offset van de bres t.o.v. kering in meter
`x2`: breedte van de bres haaks op de kering in meter
`x3`: lengte van de bres langs de kering in meter

In [ ]:
bressen.bereken_polygonen(offset=50, lengte=200, breedte=40)

## Wegschrijven resultaat

In [ ]:
bressen.to_gpkg(DATA_DIR / "resultaat.gpkg")